- references
    - https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model
    - InstructGPT: https://openai.com/index/instruction-following/

In [1]:
from IPython.display import Image

### instructGPT

In [4]:
Image(url='https://images.ctfassets.net/kftzwdyauwt9/690sZjaYwbes9IMEgfyJ7O/f536896b496b13749518f53d4caaa713/Methods_Diagram_dark_mode.jpg?w=2048&q=80&fm=webp', width=600)

- D>C>A=B

In [5]:
Image(url='./imgs/quality-labels.png', width=500)

- 单个的标注（监督学习，直接回归 rating），rating 未必是全局唯一的，
    - labeler A 认为的 3分在 labeler 的 4分或者2分，即 rating 的 scale 未必对所有的 labelers 都是适用的；
    - 李克特量表（Likert scale）
- reward model
    - 双输入，siamese network；

### Bradley-Terry model

In [3]:
Image(url='https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F0605ace1-cc21-400a-941f-154a1faa103c_1600x1427.png', width=400)